# MeetingMate - A GenAI Powered Smart Meeting Companion

Welcome to the capstone project of the Generative AI Intensive Course!  

Problem: Tired of Chaotic, Unproductive Meetings?
We’ve all been there—meetings that run long, feel unstructured, and leave you wondering what actually got decided. It’s hard to keep track of key takeaways, next steps, or even who said what. And when the meeting ends? You’re left digging through scattered notes or asking, “Wait, what did we agree on?”

Solution: Meet Your Meeting’s New Best Friend - MeetingMate.

Just upload your meeting audio, and MeetingMate takes care of the rest. It gives you a clean summary, a Q&A chatbot trained on your discussion, and even a downloadable slide deck. Built with OpenAI’s Whisper, Google Gemini, ChromaDB, and smart Python automation, MeetingMate turns your conversations into clear, actionable insights automatically.

We combine various tools such as Whisper for transcription, the Gemini API for content generation, and PowerPoint automation to create presentation-ready summaries. This notebook also introduces semantic retrieval with ChromaDB for contextual grounding.

---
## Key Features Demonstrated

- **Audio Understanding**: Transcribe meeting audio recordings using OpenAI's Whisper model.
- **Document Understanding**: Generate structured slide presentations using `python-pptx`.
- **Prompting**: Craft prompts for summarization, slide generation, and role-based insights.
- **Structured Output / JSON Mode**: Utilize `google.generativeai.types` to format and parse model outputs.
- **GenAI Evaluation**: Define rubric-based feedback using prompts to review generated outputs.
- **Embeddings & Retrieval-Augmented Generation (RAG)**: Use ChromaDB to store and retrieve prior meeting data based on semantic similarity.

---
## What We Built

MeetingMate takes in a meeting audio file and walks through the following pipeline:

1. **Transcription**: Converts meeting audio to text via Whisper.
2. **Summarization**: Uses the Gemini model to summarize the transcript.
3. **Slide Generation**: Automatically creates a PowerPoint deck summarizing key takeaways.
4. **Retrieval**: Retrieves relevant past meeting segments using ChromaDB and embeddings.
5. **Evaluation**: Grades the output using structured GenAI-based rubrics.

---

## Technologies Used

- [Whisper](https://github.com/openai/whisper) for audio transcription
- [Gemini API](https://ai.google.dev/) for text generation
- [python-pptx](https://python-pptx.readthedocs.io/) for slide creation
- [ChromaDB](https://docs.trychroma.com/) for semantic search

---

## Let’s Get Started!

Follow the notebook cells sequentially to build and customize your MeetingMate assistant.
Make sure your API keys are configured and all dependencies are installed.

In [1]:
!pip install -q git+https://github.com/openai/whisper.git python-pptx google-generativeai
!pip install -q chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [2]:
import os
import json
import uuid
import logging
import tempfile
from pathlib import Path
from typing import List, Dict, Any, Tuple
from google import genai

import whisper
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from google.genai import types, Client
from google.api_core import retry
import chromadb
from chromadb.utils.embedding_functions import EmbeddingFunction
import enum
from IPython.display import Markdown, display
from pptx.util import Inches
from PIL import Image, ImageEnhance
#from rich import print

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [4]:
#### Constants
MODEL_NAME = "gemini-2.0-flash"
EMBEDDING_MODEL = "models/text-embedding-004"

CHROMA_STORAGE_PATH = Path("./chroma_storage")
COLLECTION_NAME = "meeting_summary_collection"

PPTX_FILENAME = "Meeting_Summary.pptx"
MEETING_THEMES = {
    "team_sync": {"title": "Team Sync", "color": "#2E86C1", "bg_image": Path("/kaggle/input/meeting-images/meeting-2.jpg")},
    "project_kickoff": {"title": "Project Kickoff", "color": "#27AE60", "bg_image": Path("/kaggle/input/meeting-images/meeting-1.jpg")},
    "retrospective": {"title": "Sprint Retrospective", "color": "#E67E22", "bg_image": Path("/kaggle/input/meeting-images/meeting-4.jpg")},
    "client_review": {"title": "Client Review", "color": "#8E44AD", "bg_image": Path("/kaggle/input/meeting-images/meeting-3.jpg")},
    "default": {"title": "Meeting Summary", "color": "#34495E", "bg_image": Path("/kaggle/input/meeting-images/meeting-5.jpg")},
}

MAX_OUTPUT_TOKENS = 10000
TEMPERATURE = 0.4
TOP_P = 0.9
TOP_K = 40
EMBEDDING_TASK = "retrieval_document"

In [5]:
logger = logging.getLogger(__name__)
# Suppress httpx and google_genai.models INFO logs by default
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("google_genai.models").setLevel(logging.WARNING)

# Configure root logger
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [6]:
# set api key

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

## Upload Audio File

This utility saves uploaded audio content to a temporary `.wav` file 
for processing by the transcription model.

In [7]:
audio_dir = Path('/kaggle/input')
wav_files = [f for f in audio_dir.rglob('*') if f.suffix in ['.wav', '.mp3']]
if not wav_files:
    raise FileNotFoundError("No .wav files found in /kaggle/input")
# Consider the first audio file
src_path = wav_files[0]
print("Using audio file:", src_path)

Using audio file: /kaggle/input/audio-input/spkr0.wav


In [8]:
# save audio file
def save_file(content):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.wav') 
    temp_file.write(content)
    temp_file.close()
    return Path(temp_file.name)

## Audio Transcription (Whisper)

This class loads a Whisper model and provides transcription functionality.  
It is used to convert spoken meeting content into raw text.

In [9]:
class AudioTranscriber:
    """
    Transcribe speech to text using Whisper.
    """

    def __init__(self, model_name = "base"):
        try:
            self.model = whisper.load_model(model_name)
            logger.info(f"Loaded Whisper model '{model_name}'")
        except Exception:
            logger.exception("Failed to load Whisper model.")
            raise

    def transcribe(self, audio_path):
        """
        Transcribe the audio file and return the transcript.
        """
        try:
            result = self.model.transcribe(str(audio_path))
            logger.info("Transcription successful.")
            return result.get("text", "")
        except Exception:
            logger.exception(f"Transcription failed for {audio_path}")
            raise

## Meeting Summarization (Gemini)

This class summarizes meeting transcripts using Google's Gemini API.  
It sends a structured prompt and expects the response in JSON format, which is then parsed for downstream use like slide generation.

In [10]:
class MeetingSummarizer:
    """
    Summarizes meeting transcripts via Google GenAI.
    """

    def __init__(self, client):
        self.client = client

    def summarize(self, transcript, prompt):
        """
        Generates a structured summary (sections with titles, summary, bullets).
        """
    
        config = types.GenerateContentConfig(
            max_output_tokens=MAX_OUTPUT_TOKENS,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            top_k=TOP_K,
        )
        try:
            resp = self.client.models.generate_content(
                model=MODEL_NAME, config=config, contents=[prompt, transcript]
            )
            summary_text = resp.text
            json_str = summary_text.split("```json")[1].split("```")[0]
            summary_slides = json.loads(json_str)
            logger.info("Parsed summary to JSON.")
            return summary_slides, summary_text
        except Exception:
            logger.exception("Summarization error.")
            raise

## Evaluation with Gemini (Rubric-Based)

Here, a GenAI-based evaluator scores the generated summary for its presentation quality.

The model is prompted with an evaluation rubric based on:
- Slide structure
- Groundedness
- Conciseness
- Fluency

This enables an automated review loop that mimics human feedback for quality control.

In [11]:
class SummaryRating(enum.Enum):
    VERY_GOOD = '5'
    GOOD = '4'
    OK = '3'
    BAD = '2'
    VERY_BAD = '1'
    
class SummaryEvaluator:
    """
    Evaluates Summary generated from transcript via Google GenAI.
    """
    def __init__(self, client):
        self.client = client

    def eval(self, summary, transcript, summary_prompt):
        """
        Evaluate the summary of the transcript and provide a rating on a scale 1 to 5, 1 being "Very Poor" and 5 being "Excellent"
        """
        prompt = (f"""
            # Instruction
            You are an expert evaluator for slide presentations. Your task is to evaluate the quality of a meeting summary generated for a transcript by an AI model, which is intended to be turned into a PowerPoint slide deck.
            
            We will provide you with the original prompt and transcript given to the model and the AI-generated structured summary. Your evaluation should focus on whether this summary is effective for slide-based presentation.
            
            # Evaluation
            ## Metric Definition
            You will assess the meeting summary’s quality with regard to slide-readiness. A good slide summary should:
            - Break the content into clear sections
            - Contain accurate and concise summaries
            - Use bullet points that can be used directly in presentation slides
            - Avoid introducing information that wasn't in the source
            
            ## Criteria
            1. **Structure for Slides**: The summary is clearly broken down into presentation-friendly sections with meaningful titles.
            2. **Groundedness**: The summary uses only content grounded in the original meeting transcript and does not hallucinate.
            3. **Conciseness and Slide-Readiness**: The bullets are clear, well-chunked, and ready to be used on slides (not full paragraphs).
            4. **Fluency and Readability**: The summaries and bullets are easy to understand and grammatically correct.
            
            ## Rating Rubric
            5 (Excellent): Summary is well-structured, fully grounded, concise, and presentation-ready with fluent writing.
            4 (Good): Summary is mostly well-structured and grounded; bullets are usable with minor edits.
            3 (Fair): Summary is okay but needs editing to be usable in slides (e.g., too verbose, not well-structured).
            2 (Poor): Summary is grounded but hard to use in a slide deck without major revisions.
            1 (Very Poor): Summary is ungrounded, off-topic, or incoherent.
            
            ## Evaluation Steps
            STEP 1: Assess the summary for presentation-readiness using the 4 criteria.
            STEP 2: Score the summary using the rubric.
            
            # User Inputs and AI-generated Response
            ## Prompt
            {summary_prompt}
            ## transcript
            {transcript}
            
            ## AI-generated Summary (JSON format intended for slide generation)
            ```
            {summary}
            ```
                """
        )
        try:
            resp = self.client.chats.create(
                model=MODEL_NAME).send_message(prompt)
            verbose_eval = resp.text
            logger.info("Evaluated the summary generated.")
            return verbose_eval
        except Exception:
            logger.exception("Evaluation error.")
            raise


## Slide Generation (python-pptx)

This section uses the `python-pptx` library to automatically convert the structured JSON summary 
into a clean and readable PowerPoint presentation. It includes:
- Section titles
- Summaries
- Bullet points styled for readability

The goal is to reduce manual effort in summarizing and preparing meeting notes into presentable slides.

In [12]:
class PPTGenerator:
    """
    Create a PowerPoint from structured summary.
    """

    def __init__(self, themes):
        self.themes = themes

    @staticmethod
    def hex_to_rgb(h):
        h = h.lstrip("#")
        return tuple(int(h[i : i + 2], 16) for i in (0, 2, 4))

    @staticmethod
    def is_dark(rgb):
        r, g, b = rgb
        return (0.299 * r + 0.587 * g + 0.114 * b) < 150

    @staticmethod
    def add_bg(slide, rgb, image_path):
        """
        Adds a background image or fallback color to the slide.
        """
        try:
            if image_path:
                # Resize image first
                with Image.open(image_path) as img:
                    img = img.resize((960, 720))  # PowerPoint slide size
                    temp_path = "/tmp/resized_bg.jpg"
                    img.save(temp_path)
    
                # Apply opacity to resized image
                faded_path = "/tmp/faded_bg.png"
                PPTGenerator.apply_opacity(temp_path, faded_path, opacity=0.3)
    
                # Add faded image to slide
                img_shape = slide.shapes.add_picture(faded_path, 0, 0, width=Inches(10), height=Inches(7.5))
            
                # Send to back of z-order
                spTree = slide.shapes._spTree
                spTree.remove(img_shape._element)
                spTree.insert(2, img_shape._element)
                return
        except Exception:
            logger.exception("Background image failed. Falling back to color.")
    
        # Fallback to solid color background
        if rgb:
            shape = slide.shapes.add_shape(
                MSO_SHAPE.RECTANGLE, Inches(0), Inches(0), Inches(10), Inches(7.5)
            )
            shape.fill.solid()
            shape.fill.fore_color.rgb = RGBColor(*rgb)
            shape.line.fill.background()
            slide.shapes._spTree.insert(2, slide.shapes._spTree[-1])
    @staticmethod
    def apply_opacity(image_path, output_path, opacity = 0.3):
        """
        Saves a faded version of the image to use as background.
        """
        img = Image.open(image_path).convert("RGBA")
        alpha = img.split()[3]
        alpha = ImageEnhance.Brightness(alpha).enhance(opacity)
        img.putalpha(alpha)
        img.save(output_path)

    def generate(self, sections, filename, mtype = "default"):
        """
        Build and save the .pptx file.
        """
        try:
            theme = MEETING_THEMES.get(mtype, MEETING_THEMES["default"])
            rgb = self.hex_to_rgb(theme["color"])
            bg_image = theme.get("bg_image")
            font_rgb = (0, 0, 0)
            use_light_text = self.is_dark(rgb)
            prs = Presentation()

            # Title Slide
            slide = prs.slides.add_slide(prs.slide_layouts[0])
            self.add_bg(slide, rgb=rgb, image_path=bg_image)
            slide.shapes.title.text = theme["title"]
            slide.shapes.title.text_frame.paragraphs[0].font.color.rgb = RGBColor(
                *font_rgb
            )
            body = slide.placeholders[1]
            body.text = filename.stem
            body.text_frame.paragraphs[0].font.color.rgb = RGBColor(*font_rgb)

            # TOC
            toc = prs.slides.add_slide(prs.slide_layouts[1])
            self.add_bg(toc, rgb=rgb, image_path=bg_image)
            toc.shapes.title.text = "Table of Contents"
            toc.shapes.title.text_frame.paragraphs[0].font.color.rgb = RGBColor(
                *font_rgb
            )
            toc_body = toc.placeholders[1]
            toc_body.text = "\n".join(s["section_title"] for s in sections)
            for p in toc_body.text_frame.paragraphs:
                p.font.color.rgb = RGBColor(*font_rgb)

            # Content Slides
            for sec in sections:
                sld = prs.slides.add_slide(prs.slide_layouts[1])
                self.add_bg(sld, rgb=rgb, image_path=bg_image)
                sld.shapes.title.text = sec["section_title"]
                sld.shapes.title.text_frame.paragraphs[0].font.color.rgb = RGBColor(
                    *font_rgb
                )
                box = sld.placeholders[1]
                tf = box.text_frame
                tf.clear()
                summary_text = sec.get("summary", "")
                summary_pt = tf.add_paragraph()
                summary_pt.text = f"Summary: {summary_text}"
                summary_pt.font.size = Pt(18)
                summary_pt.font.color.rgb = RGBColor(*font_rgb)
                summary_pt.font.bold = True
                for bullet in sec.get("bullets", []):
                    pb = tf.add_paragraph()
                    pb.text = bullet
                    pb.level = 1
                    pb.font.size = Pt(20)
                    pb.font.color.rgb = RGBColor(*font_rgb)
                    

            prs.save(str(filename))
            logger.info(f"PPT saved: {filename}")
        except Exception:
            logger.exception("Failed PPT generation.")
            raise

## Embedding and Retrieval

This part introduces **retrieval-augmented generation (RAG)** by embedding previous meeting transcripts 
and storing them in a vector database (`ChromaDB`). When a new meeting is processed, similar 
past meetings can be retrieved to provide context or track recurring themes.

This enhances the agent’s memory and understanding of long-term patterns.

In [13]:
class RAGEngine:
    """
    Retrieval-Augmented Generation using ChromaDB.
    """
    def __init__(self, client, storage_path, collection_name):
        self.client = client
        self.storage_path = storage_path
        self.collection_name = collection_name

    class _EmbeddingFn(EmbeddingFunction):
        def __init__(self, client, model):
            self.client = client
            self.model = model

        def __call__(self, texts):
            try:
                res = self.client.models.embed_content(
                    model=self.model,
                    contents=texts,
                    config=types.EmbedContentConfig(
                    task_type=EMBEDDING_TASK,
                    ),
                )
                raw = getattr(res, 'embeddings', None) or res.get('embeddings') or res.get('embedding')
                if not raw:
                    raise ValueError("No embeddings returned from GenAI.")
                processed = [item.values if hasattr(item, 'values') else item for item in raw]
                return processed
            except Exception:
                logger.exception("Embedding failed.")
                return []

    def init_db(self) :
        """
        Initialize or get a ChromaDB collection.
        """
        try:
            client = chromadb.PersistentClient(path=str(self.storage_path))
            existing = [col.name for col in client.list_collections()]
            if self.collection_name in existing:
                logger.info("Using existing ChromaDB collection.")
                return client.get_collection(name=self.collection_name)
            logger.info(f"Creating new ChromaDB collection: {self.collection_name}")
            return client.create_collection(
                name=self.collection_name,
                embedding_function=self._EmbeddingFn(self.client, EMBEDDING_MODEL)
            )
        except Exception:
            logger.exception("Failed to initialize ChromaDB collection.")
            raise

    def add_document(self, db, text):
        """
        Add a document to ChromaDB.
        """
        try:
            if text:
                db.add(documents=[text], ids=[str(uuid.uuid4())])
                logger.info("Document added to ChromaDB.")
        except Exception:
            logger.exception("Failed to add document to ChromaDB.")

    def query(self, db, query, k = 2):
        """
        Query ChromaDB for top-k relevant documents.
        """
        try:
            result = db.query(query_texts=[query], n_results=k)
            return result.get('documents', [])
        except Exception:
            logger.exception("Failed to query ChromaDB.")
            return []

    def answer(self, db, query, k = 2):
        """
        Answer a question using retrieved passages from ChromaDB.
        """
        try:
            passages = self.query(db, query, k)
            prompt = (
                f"""You are a helpful and informative bot that answers questions using only the
                provided reference passage below.
                
                Instructions:
                - Provide a complete, well-explained answer based solely on the passage.
                - If the answer is not available, respond with: "I'm not sure."
                - You are responding to a technical audience, so explain clearly but concisely.
                - Break down complex concepts into understandable parts.
                - Ignore irrelevant information.
                
                Passage:
                {passages}
                
                Question:
                {query}

                Answer:RAG Q&A"""
            )
            resp = self.client.models.generate_content(
                model=MODEL_NAME,
                contents=[prompt]
            )
            return resp.text.strip()
        except Exception:
            logger.exception("Failed to generate answer.")
            return "I'm not sure."

## End-to-End Workflow: Transcribe → Summarize → Evaluate → Generate Slides → Retrieve → QA

This function orchestrates the full **MeetingMate pipeline**:

1. **Transcribe**: Converts raw audio to text using Whisper.
2. **Summarize**: Uses Gemini API with structured prompting to generate a JSON summary.
3. **Evaluate**: Scores the summary using a rubric-based GenAI evaluator.
4. **Detect Theme**: Classifies the meeting type (e.g., retrospective, kickoff).
5. **Generate Slides**: Converts the summary into a PowerPoint presentation.
6. **RAG Q&A**: Embeds the transcript and performs retrieval-augmented question answering using ChromaDB.

This design demonstrates a complete GenAI application combining multimodal processing, structured prompting, evaluation, generation, and retrieval.

In [14]:
def main(audio_bytes):
    """
    Workflow: transcribe -> summarize and RAG Q&A -> detect type -> ppt
    """
    try:
        key = GOOGLE_API_KEY
        gen_client = Client(api_key=key)

        #### Transcription ####
        transcriber = AudioTranscriber()
        path = save_file(audio_bytes)
        transcript = transcriber.transcribe(path)

        #### Summarization ####
        summary_prompt = ("""
            You are a meeting assistant. Carefully analyze the meeting transcript and:
                1. Segment it into distinct topics (whenever the conversation focus shifts).
                2. For each topic:
                  - Assign a meaningful short section_title (max 1 line).
                  - Write a concise 1-sentence summary.
                  - Extract 2-3 bullet points (each bullet under 50 words).
                
            Think step by step. Identify topic shifts chronologically.
            Return the result strictly in the JSON format. All keys must be in double quotes:
                ```
                  {
                    "section_title": "Team Updates",
                    "summary": "...",
                    "bullets": ["...", "...", "..."]
                  },
                  ...
                ```
                """
        )
        summarizer = MeetingSummarizer(gen_client)
        summary, summary_text = summarizer.summarize(transcript, summary_prompt)
        display(Markdown("### **1. Structured output of the Summary**"))
        display(Markdown(summary_text))

        #### Summary Evaluation ####
        evaluate = SummaryEvaluator(gen_client)
        evaluation = evaluate.eval(summary, transcript, summary_prompt)
        display(Markdown(f"### 2. Rubric-Based Evaluation\n{evaluation}"))
        
        #### Theme Detection ####
        theme_prompt = ("""
        You're a Meeting assistant. Given the context, understand it and choose the meeting type accordingly from the list:
        ["team_sync", "project_kickoff", "client_review", "retrospective"].
                
        Reply with just the meeting type.
        """
        )
        theme_resp = gen_client.models.generate_content(
            model=MODEL_NAME, contents=[theme_prompt, transcript]
        )
        mtype = theme_resp.text.strip().lower()
        display(Markdown("### **3. Meeting Type**"))
        display(Markdown(mtype))

        #### PPT generation ####
        ppt = PPTGenerator(MEETING_THEMES)
        ppt.generate(summary, Path(PPTX_FILENAME), mtype)

        #### RAG Q&A using Chroma ####
        rag = RAGEngine(gen_client, CHROMA_STORAGE_PATH, COLLECTION_NAME)
        db = rag.init_db()
        rag.add_document(db, transcript)


        # Example QA
        questions = [
            "What's the main priority for getting people to the underused room?",
            "What audience groups are excluded?",
            "What's the plan for distributing audio recordings or 'CDs' post-session?",
        ]

        display(Markdown("### **4. RAG Q&A**"))
        for question in questions:
            display(Markdown("Q:" + question))
            ans = rag.answer(db, question, k=2)
            display(Markdown("Ans:"+ans))

    except Exception:
        logger.exception("Main workflow error.")
        raise
    display(Markdown("### **Done**"))


if __name__ == "__main__":
    audio_bytes = src_path.read_bytes()
    main(audio_bytes)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 194MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

### **1. Structured output of the Summary**

```json
[
  {
    "section_title": "Logistics of In-Person Meetings and Participant Incentives",
    "summary": "The team discusses strategies to encourage attendance at in-person meetings, including offering incentives like transcripts and CDs of participants' speech, while also addressing concerns about data privacy and resource limitations.",
    "bullets": [
      "The team explores the idea of providing participants with a CD of their speech as an incentive, but raises concerns about privacy and data usage.",
      "Offering transcripts after a screening phase is suggested as a compromise to address privacy concerns while still providing value to participants.",
      "The team emphasizes the importance of recruiting a diverse group of participants, including those outside of linguistics and engineering."
    ]
  },
  {
    "section_title": "Disk Space Management and Archiving Progress",
    "summary": "The team provides an update on the progress of archiving files to free up disk space for recording future meetings, highlighting the time-consuming nature of the process and the benefits of having archived data readily accessible.",
    "bullets": [
      "Archiving old files, particularly from broadcast news, is underway to create more disk space for recording meetings.",
      "The archiving process is time-intensive, but it allows for quick retrieval of data when needed.",
      "After the current archiving is complete, the team will have enough space to record approximately five more meetings."
    ]
  }
]
```

### 2. Rubric-Based Evaluation
## Evaluation
Here's an evaluation of the AI-generated summary based on the criteria:

**STEP 1: Assessment**

*   **Structure for Slides**: The summary is well-structured, dividing the content into two distinct sections with clear and meaningful titles.
*   **Groundedness**: The summary is grounded in the original meeting transcript. All points discussed are present in the source material.
*   **Conciseness and Slide-Readiness**: The bullet points are concise and suitable for slides. They effectively summarize key points from the transcript.
*   **Fluency and Readability**: The summaries and bullet points are easy to understand and grammatically correct.

**STEP 2: Score**

Based on the assessment, the summary is rated as **5 (Excellent)**. It is well-structured, fully grounded, concise, presentation-ready, and fluently written.


### **3. Meeting Type**

...

Q:What audience groups are excluded?

Ans:The audience groups that are excluded are linguists and engineers because a wider sampling of people is needed.

Q:What's the plan for distributing audio recordings or 'CDs' post-session?

Ans:Initially, there was discussion about providing participants with a CD of their speech immediately after the session. However, concerns were raised regarding the content of these recordings and whether they were suitable for immediate release. The revised plan involves a two-step process: first, transcripts of the recordings will be screened. After this screening phase, participants will receive a CD. It has been specified that the CD given to the participants should be the same version that has been reviewed publicly to ensure compliance.

### **Done**